# Fit terrain mesh from rover images


In [ ]:
import os
import numpy as np

from pytorch3d.vis.plotly_vis import plot_scene

from pytorch3d_utils import structured_grid_to_pytorch3d_mesh
from lac.utils.plotting import plot_poses
from lac.util import load_data

In [ ]:
map = np.load("../../../data/heightmaps/qualifying/Moon_Map_01_preset_0.dat", allow_pickle=True)

mesh = structured_grid_to_pytorch3d_mesh(map[..., :3])

In [ ]:
# Render the plotly figure
fig = plot_scene({"subplot1": {"map_mesh": mesh}})
fig.update_layout(width=1200, height=700, scene=dict(aspectmode="cube"))
fig.update_layout(
    scene=dict(
        xaxis=dict(showgrid=True, showbackground=True),
        yaxis=dict(showgrid=True, showbackground=True),
        zaxis=dict(showgrid=True, showbackground=True),
    )
)
fig.show()

In [ ]:
data_path = "../../../output/LocalizationAgent/map1_preset0_4m_spiral"
initial_pose, lander_pose, poses, imu_data, cam_config = load_data(data_path)

In [ ]:
plot_poses(poses[::100])

In [ ]:
# Render the plotly figure
fig = plot_scene({"subplot1": {"map_mesh": mesh}})
# fig.update_layout(width=1200, height=700, scene=dict(aspectmode="cube"))
fig = plot_poses(poses[::100], fig=fig)
fig.update_layout(width=1200, height=700, scene=dict(aspectmode="data"))
fig.show()

In [ ]:
import cv2
import matplotlib.pyplot as plt

In [ ]:
step = 6500

FL_gray = cv2.imread(os.path.join(data_path, "FrontLeft", f"{step}.png"), cv2.IMREAD_GRAYSCALE)

plt.imshow(FL_gray, cmap="gray")
plt.show()

In [ ]:
from lac.utils.frames import get_cam_pose_rover
from lac.utils.frames import CAMERA_TO_OPENCV_PASSIVE

rover_pose = poses[6500]
camera_pose = get_cam_pose_rover("FrontLeft")
camera_pose[:3, :3] = CAMERA_TO_OPENCV_PASSIVE
camera_pose = rover_pose @ camera_pose

In [ ]:
import torch

from pytorch3d.renderer import (
    PointLights,
    FoVPerspectiveCameras,
    RasterizationSettings,
)

if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.cuda.set_device(device)
else:
    device = torch.device("cpu")

In [ ]:
# Place a point light in front of the object. As mentioned above, the front of
# the cow is facing the -z direction.
lights = PointLights(device=device, location=[[0.0, 0.0, -3.0]])

In [ ]:
num_views = 1000
R = []
T = []
target_rgb = []

idxs = np.arange(10, 1000, 10)

for i in idxs:
    R.append(torch.tensor(poses[i][:3, :3], device=device).float())
    T.append(torch.tensor(poses[i][:3, 3], device=device).float())
    img_np = cv2.imread(os.path.join(data_path, "FrontLeft", f"{i}.png"))
    new_size = (img_np.shape[1] // 2, img_np.shape[0] // 2)
    downscaled_img_np = cv2.resize(img_np, new_size, interpolation=cv2.INTER_AREA)
    target_rgb.append(torch.tensor(downscaled_img_np, device=device).float())

R = torch.stack(R)
T = torch.stack(T)

In [ ]:
target_rgb[0].shape

In [ ]:
cameras = FoVPerspectiveCameras(device=device, R=R, T=T)

In [ ]:
raster_settings = RasterizationSettings(
    image_size=(360, 640),
    blur_radius=0.0,
    faces_per_pixel=1,
)